In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

from transformers import BertForMaskedLM, BertTokenizerFast
import torch
import copy
model = BertForMaskedLM.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
tokenizer = BertTokenizerFast.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
MAX_SEQ_LEN = 85
from mlm_utils.model_utils import TOKENIZER

import torch.nn.functional as F
from mlm_utils.preprocess_functions import get_key, compare_tensors, get_pos_tag_word


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-04-02 10:57:41.606717: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 10:57:42.102198: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 10:57:44.526981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-02 10:57:47.821555: E tensorflow/compiler/xla/stream_executor/cuda/cu

## test mask content wword

In [9]:
from prepared_for_mlm import decode_token, get_tokens_for_words, mask_content_words
def mask_content_words(ids, word_dict):
    '''
    input:
        ids: sample_id 
        word_dict: {'The': [tensor(170)], 'capital': [tensor(1109), tensor(3007)], ....}
    output:
        masked_sentences: [tensor([  101,  1103, 175, 10555,  103,   103,   103,   102])
        label_ids: [tensor(6468)], [tensor(1568), tensor(13892)]
    '''
    labels = []
    masked_sentences = []
    except_tokens = [TOKENIZER.cls_token_id, TOKENIZER.sep_token_id, TOKENIZER.pad_token_id, TOKENIZER.unk_token_id]
    count_masked_words = []
    for (key, value) in word_dict.items():
        masked_ids = ids.clone() # torch.Size([1, 85])
        origin_sample = decode_token(masked_ids[0])
        if get_pos_tag_word(key, origin_sample).values in ['NOUN', 'VERB', 'ADJ', 'ADV']:
            print("KEY: ", key)
            for i in range(len(masked_ids[0]) - len(value) + 1):
                masked_id = masked_ids.clone()
                label = torch.full_like(masked_id, fill_value=-100)
                masked_indice = torch.full_like(masked_id, fill_value=0)
            
                if torch.equal(torch.as_tensor(masked_id[0][i:i+len(value)]).clone().detach(), torch.as_tensor(value).clone().detach()):
                    masked_id[0][i:i+len(value)]= TOKENIZER.mask_token_id
                    #print("MASKED ID: ", masked_id)
                    masked_indice[masked_id == TOKENIZER.mask_token_id] = 1
                    #print("MASKED INDICE: ", masked_indice)
                    
                    
                    for idx, mask in enumerate(masked_indice[0]):
                        if mask == 1:
                            label[0][idx] = ids[0][idx]
                            
                    masked_sentences.append(masked_id)
                    labels.append(label)
                # count_masked_words.append(count_masked_key)
            
    return masked_sentences, labels

origin_input_id = torch.tensor([  101,   189, 27226,  1116,  1114,  2549, 21521,  1198,   126,  1106,
         4252,  1320,   122,   117,  1137,  6531,  1121,   172,   118,  1139,
         1665,   117,  1138,  2999,   172,   118,  1139,  1665,   182, 11782,
          1116,  1104,   123,   119,  1512,  1105,   123,   119,   125,   180,
         1830,   117,  1134, 11271,  1120,  1147,   126,  3769,   117,  1229,
         189, 27226,  1116,  1114,  2549, 21521,  1439,  4252,  1320,   122,
         1137, 27553,  1179,   122, 13000,   172,   118,  1139,  1665,   182,
        11782,   1116,   113,   123,   119,   122,   118,   123,   119,   128,
          180,  1830,  1107,   171,   102])

origin_text = tokenizer.decode_plus(origin_input_id, skip_special_tokens=True, return_offsets_mapping=True) 
print(origin_text)
doc = nlp(origin_text)
words_str = [word.text for word in [token for token in doc]]

token_test = tokenizer.encode_plus(
                ' '.join(words_str),
                max_length=MAX_SEQ_LEN,
                padding='max_length', 
                truncation=True,  
                add_special_tokens = True,
                return_tensors="pt",  
                return_attention_mask = True,
                return_offsets_mapping=True  
            )

word_dict = get_tokens_for_words(
    words_str, 
    token_test['input_ids'], 
    token_test['offset_mapping'][0]
    )

masked_sen_2d = origin_input_id.view(-1, 85)

masked_sentences, label_ids = mask_content_words(masked_sen_2d, word_dict)
# print(len(masked_sentences), len(label_ids  ))
# for i in range(len(masked_sentences)):
#     print(label_ids[i], masked_sentences[i])


AttributeError: 'BertTokenizerFast' object has no attribute 'decode_plus'

## test is POs match


In [2]:
def get_pos_tag_word(word, text):
    doc = nlp(text)
    word_split = word.split()
    print("================")
    for token in doc:
        if token.text in word_split:
            print(token.text, token.pos_)
        #     return token.pos_
    return None

def get_key(dictionary, value, count_word):
    unique_set = torch.chunk(value, count_word)[0]
    for key, val in dictionary.items():
        if compare_tensors(val, unique_set):
            return key
    return None 
def is_POS_match(input_ids, lm_label_ids, b_count_word):
    '''
    Function to check if the POS tag of the masked token in the logits is the same as the POS tag of the masked token in the original text.
    Note: This function assumes that the logits are of shape # ([85, 28996]) 
    lm_label_ids: shape (batch_size, sequence_length)
    '''
    
    origin_input_id = input_ids.clone() # Origin input id:  torch.Size([85])
    result_ids = input_ids.clone() # torch.Size([85])
    
    # Find the index of the masked token from lm_label_ids
    masked_idx = torch.where(lm_label_ids != -100)[0]
    masked_idx_input = torch.where(input_ids == tokenizer.mask_token_id)[0]
    origin_input_id[masked_idx_input] = lm_label_ids[masked_idx] 
    
    
    # make sure masked_idx_input and masked_idx are the same using assert
    assert torch.equal(masked_idx, masked_idx_input), "Masked index and label index are not the same."
    print("lm_label_ids[masked_idx]  ", lm_label_ids )
    # get pos tag of origin text
    origin_text = tokenizer.decode(origin_input_id, skip_special_tokens=True) 
    
    doc = nlp(origin_text)
    words_str = [word.text for word in [token for token in doc]]
    
    token_test = tokenizer.encode_plus(
                    ' '.join(words_str),
                    max_length=MAX_SEQ_LEN,
                    padding='max_length', 
                    truncation=True,  
                    add_special_tokens = True,
                    return_tensors="pt",  
                    return_attention_mask = True,
                    return_offsets_mapping=True  
                )
    masked_sens_2d = torch.unsqueeze(input_ids, 0)
    label_ids_2d = torch.unsqueeze(lm_label_ids, 0)
    output = model(input_ids = masked_sens_2d, attention_mask = token_test['attention_mask'], token_type_ids=token_test['token_type_ids'], labels=label_ids_2d)
    print("shape logits: ", output.logits.shape)  ##shape logits:  torch.Size([1, 85, 28996])
    
    word_dict = get_tokens_for_words(
        words_str, 
        token_test['input_ids'], 
        token_test['offset_mapping'][0]
        )
      
    # Get key for the masked token
    
    masked_word = get_key(word_dict, origin_input_id[masked_idx],b_count_word )
    print("masked word: ", masked_word)
    pos_tag_origin = get_pos_tag_word(masked_word, origin_text)
    print("pos tag origin: ", masked_word, pos_tag_origin)         
    
    # # Extract the logits for the masked position
    masked_logits = output.logits[0][masked_idx]
    # print("MASKED LOGITS: ", masked_logits) # torch.Size([28996])
    pred = [torch.argmax(output.logits[0][i]).item() for i in masked_idx]
    # Print top 10 masked tokens
        # print(tokenizer.convert_ids_to_tokens(torch.topk(outputs.logits[0, idx, :], 10).indices))
        
    # Replace the index of the masked token with the list of predicted tokens
    for i in range(len(masked_idx)):
        result_ids[masked_idx[i]] = pred[i]
        
    # print("result sentence: ", tokenizer.decode(result_ids, skip_special_tokens=True))
    for i in pred:
        print("PRED WORD: ",i,  tokenizer.decode(i))
    print("PRED WORD: ",pred,  tokenizer.decode(pred))
    logits_tag = get_pos_tag_word(tokenizer.decode(pred), tokenizer.decode(result_ids, skip_special_tokens=True) )
    print("POS TAG PRED WORD: ", logits_tag)
    
    
    # Cross-entropy term
    
    cross_entropy_term = F.cross_entropy(output.logits.view(-1, tokenizer.vocab_size), lm_label_ids.view(-1))
    print("Cross entropy term shape: ", cross_entropy_term.shape)      ##Cross entropy term shape:  torch.Size([2720])
  
    # Custom matching term
    matching_term_batch = (pos_tag_origin == logits_tag)

    # Combine terms to get the loss for 1 batch
    matching_term = torch.where(matching_term_batch == torch.tensor(True), torch.tensor(1.0), torch.tensor(0.0))
    print("Matching term shape: ", matching_term)  ##Matching term shape:  torch.Size([2720])
    print("cross entropy term: ", cross_entropy_term)
    
    loss = 0.5 * cross_entropy_term + (1 - matching_term)
    
    
    return loss

In [3]:
label_id = torch.tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,   188,  1643, 22548,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100])

input_id= torch.tensor([  101,  1142,  2848,  3792,  2217,   118,   171,  1665,  1918,   187,
         1605,  1108, 23448, 11549,  1193, 14715, 13098,  1121,  1103,  1269,
        25338,  6697,  1112,   171,  1665,  1918,   117,  1105, 10877,   182,
        11782,  7987,  1956,   117,   174,   119,   176,   119,   185, 23415,
         6397,  3382,  6840,  1105,   103,   103,   103,   119,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])     

print(is_POS_match(input_id, label_id, 2))

lm_label_ids[masked_idx]   tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, 5190, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, 5190, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100])
shape logits:  torch.Size([1, 85, 28996])
masked word:  acid
acid NOUN
acid NOUN
pos tag origin:  acid None
PRED WORD:  26825 insulin
PRED WORD:  3850 drug
PRED WORD:  [26825, 3850] insulin drug
insulin NOUN
drug NOUN
POS TAG PRED WORD:  None
Cross entropy term shape:  torch.Size([])
Matching term shape:  tensor(1.)
cross entropy term:  tensor(4.9894, grad_fn=<NllLossBackward0>)
tensor(2.4947, g

In [50]:
import torch

# Sample word dictionary and tensor
word_dict = {'tumours': [torch.tensor(189)], 'with': [torch.tensor(1114)]}
given_tensor = torch.tensor([189, 27226, 1116, 189, 27226, 1116])

# Function to check if any tensor in the dictionary values is in the given tensor
def is_in_tensor(word_dict, tensor):
    for word, tensors in word_dict.items():
        for tensor_val in tensors:
            if (tensor == tensor_val).any().item():
                print(f"{word} is in the given tensor.")
                break

# Check if any word in the dictionary is in the given tensor
is_in_tensor(word_dict, given_tensor)


tumours is in the given tensor.


In [4]:
import torch

# Assuming logits is your tensor of shape (batch_size, max_seq_length, vocab_size)
logits = torch.randn((1, 5, 10))  # Example tensor, replace it with your actual logits

# Compute the inverse of the logits
inverse_logits = 1 - logits

# You can also use torch.sub() function
# inverse_logits = torch.sub(1, logits)
print(logits)
print(inverse_logits)  # Should output the same shape as logits


tensor([[[-0.7282,  0.6782, -0.3622,  0.9572,  0.9279,  1.0818, -0.4334,
          -0.8105, -0.1358,  0.5705],
         [ 1.3758, -2.1371, -0.0726, -1.8334, -1.3297, -0.1785,  1.5264,
           1.5781,  0.3917, -2.0859],
         [ 0.6384,  0.3851, -1.4952,  1.9068, -0.4026, -0.4253, -0.2667,
          -0.0509,  0.0526,  0.5777],
         [ 0.4248, -0.0755, -0.2062,  0.8857, -0.0867,  0.0920, -1.0359,
           0.9809,  2.1640,  0.6484],
         [ 0.3108, -0.1413,  0.3952, -0.2666,  1.1596, -0.7127, -0.0294,
           0.6728,  0.4344, -0.8212]]])
tensor([[[ 1.7282,  0.3218,  1.3622,  0.0428,  0.0721, -0.0818,  1.4334,
           1.8105,  1.1358,  0.4295],
         [-0.3758,  3.1371,  1.0726,  2.8334,  2.3297,  1.1785, -0.5264,
          -0.5781,  0.6083,  3.0859],
         [ 0.3616,  0.6149,  2.4952, -0.9068,  1.4026,  1.4253,  1.2667,
           1.0509,  0.9474,  0.4223],
         [ 0.5752,  1.0755,  1.2062,  0.1143,  1.0867,  0.9080,  2.0359,
           0.0191, -1.1640,  0.3516],

In [8]:
import torch

def pos_tag_mapping(pos_tag):
    if pos_tag =="NOUN":
        return 1
    elif pos_tag =="VERB":
        return 2
    elif pos_tag =="ADJ":
        return 3
    elif pos_tag == "ADV":
        return 4
    else:
        return -1

def get_pos_tag_id(word_dict, pos_tag_dict, label_id):
    pos_tag_id = []
    for label in label_id:
        word = None
        for key, value in word_dict.items():
            if label in value:
                word = key
                break
        if word is not None:
            pos_tag = pos_tag_dict.get(word)
            pos_tag_id.append(pos_tag_mapping(pos_tag))
        else:
            pos_tag_id.append(-1)
    return pos_tag_id

# Example usage
import torch

word_dict = {"Ba": torch.tensor([104, 4394]), "masture": torch.tensor([424]), "ofs": torch.tensor([434]), "a": torch.tensor([32])}
pos_tag_dict = {"Ba": "NOUN", "masture": "NOUN", "ofs":"ADJ", "a":"PUNCT"}
label_id = torch.tensor([102, 104, 4394, 424, 434, 32, -100])

token_ids = get_pos_tag_id(word_dict, pos_tag_dict, label_id)
print(token_ids)  # Output: [-1, 1, 1, 1, 3, -1, -1]




[-1, 1, 1, 1, 3, -1, -1]


In [2]:
import torch

def check_matching_terms(pred, original, mask_index, pos_tag_id_pred, pos_tag_id_origin):
    matching_term_tensor = torch.where(pos_tag_id_pred[mask_index] == pos_tag_id_origin[mask_index], 
                                       torch.tensor(1), 
                                       torch.tensor(0))
    return matching_term_tensor

# Khởi tạo các giá trị mẫu
mask_index = [0, 1, 2]  # Ví dụ danh sách các index của mask
pos_tag_id_pred = torch.tensor([1, 2, 3, 4, 5])  # Ví dụ tensor pos_tag_id_pred
pos_tag_id_origin = torch.tensor([1, 2, 3, 7, 8])  # Ví dụ tensor pos_tag_id_origin

# Kiểm tra matching terms
matching_term_tensor = torch.zeros_like(pos_tag_id_pred)
matching_term_tensor[mask_index] = check_matching_terms(pred=None, original=None, mask_index=mask_index, 
                                             pos_tag_id_pred=pos_tag_id_pred, 
                                             pos_tag_id_origin=pos_tag_id_origin)

print("Matching Term Tensor:", matching_term_tensor)


Matching Term Tensor: tensor([1, 1, 1, 0, 0])


In [29]:
pos_tag_dict = {'probed': 'VERB'}
pred_id = torch.tensor([  101,  1122,  1180,  3510, 10936,  1116,  1107,   189,  4832,  1204, 113,   189,  4889,   118, 20232,  1174, 21718,  2568,   117,   121, 119,  4991,   110,   189,  7921,  1424,   118,  406,   114,  4051, 126,   110,   171,  3292,  2042, 23651,  1312,  1394,   113,  1111, 2848,   118,   185, 15342,  7880,  4649,  5864,  2042,   171,  7841,  1116,   114,  1137, 10458, 12140,  1181,  6831,  1105, 17357, 1181,
1114, 26491,   119,   102,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,   0,     0,     0,     0,     0])

In [48]:
def pos_tag_mapping(pos_tag):
    if pos_tag =="NOUN":
        return 1
    elif pos_tag =="VERB":
        return 2
    elif pos_tag =="ADJ":
        return 3
    elif pos_tag == "ADV":
        return 4
    else:
        return -1
    
    
def get_pos_tag_id(word_dict, pos_tag_dict, label_id):
    pos_tag_id = torch.full_like(label_id, fill_value=-1)
    
    for key in pos_tag_dict.keys():
        tokens = word_dict.get(key)
        
        for i in range(len(label_id) - len(tokens) + 1):
            if torch.equal(torch.as_tensor(label_id[i:i+len(tokens)]).clone().detach(), torch.as_tensor(tokens).clone().detach()):
               
                pos_tag_id[i:i+len(tokens)] = pos_tag_mapping(pos_tag_dict.get(key))
 
    return pos_tag_id

In [49]:
list_id = get_pos_tag_id(word_dict, pos_tag_dict, pred_id)
# convert list to string
str(list_id)

tensor([17357,  1181]) tensor([17357,  1181])
58 tensor([2, 2])


'tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,\n        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,\n        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,\n        -1, -1, -1, -1,  2,  2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,\n        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])'

In [39]:
print(word_dict.get('probed'))

tensor([17357,  1181])
